## Model Training

Let's start off by loading our required libraries.

In [1]:
import pandas as pd 
import keras
import tensorflow as tf
from keras import models, layers
keras.__version__

Using TensorFlow backend.


'2.2.2'

Let's read our dataset which is a feature array which describes which games a user has previously played, and the output is the likelihood of the player to play a specific game in the future. 

In [2]:
df = pd.read_csv("https://github.com/bgweber/Twitch/raw/master/Recommendations/games-expand.csv")
df.to_csv("games.csv", index=False)
df.head()

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,label
0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0
3,0,0,1,0,0,1,1,0,0,1,1
4,0,0,1,0,1,1,0,1,1,0,1


Now let's keep first 5000 data points as Testing data and rest as training. We will also split our data into features `X` and labels `y`.

In [3]:
train = df[5000:]
test = df[:5000]

X = train.drop(['label'], axis=1)
y = train['label']

Now we will start by defining our network architecture. 

In [4]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10,)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

After defining our accuracy method and then compiling it, we fit our model.

In [5]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    keras.backend.get_session().run(tf.local_variables_initializer())
    return auc
    
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=[auc])


history = model.fit(X,
                    y,
                    epochs=100,
                    batch_size=100,
                    validation_split = .2,
                    verbose=0)

Let's see how our model performs against our test data.

In [6]:
x_test = test.drop(['label'], axis=1)
y_test = test['label']

results = model.evaluate(x_test, y_test, verbose = 0)
print(results)

[0.3832338626861572, 0.819283809375763]


Now we will save our model which we can use while deployment.

In [8]:
model.save("games.h5")

We can see that our model has been saved in our current working directory.

In [10]:
!ls

games.csv  games.h5  Model.ipynb
